In [2]:
%%configure -f
{ 
    "name": "mmlspark",
    "conf": {
        "spark.jars.packages": "com.microsoft.ml.spark:mmlspark_2.11:1.0.0-rc1",
        "spark.jars.repositories": "https://mmlspark.azureedge.net/maven",
        "spark.jars.excludes": "org.scala-lang:scala-reflect,org.apache.spark:spark-tags_2.11,org.scalactic:scalactic_2.11,org.scalatest:scalatest_2.11",
        "spark.pyspark.python": "python3",
        "spark.pyspark.virtualenv.enabled": "true",
        "spark.pyspark.virtualenv.type":"native",
        "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
    }
}

In [3]:
# sc.install_pypi_package("mmlspark")
sc.install_pypi_package("matplotlib")
# sc.install_pypi_package("ipympl")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
9,application_1589294896777_0010,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached matplotlib-3.2.1-cp36-cp36m-manylinux1_x86_64.whl (12.4 MB)
  Using cached pyparsing-2.4.7-py2.py3-none-any.whl (67 kB)
  Using cached python_dateutil-2.8.1-py2.py3-none-any.whl (227 kB)
  Using cached kiwisolver-1.2.0-cp36-cp36m-manylinux1_x86_64.whl (88 kB)
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)

In [12]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
# from pyspark.sql.functions import *
import pyspark.sql.functions as F
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import col
from pyspark.sql.functions import round
from pyspark.sql.types import StringType
from pyspark import SQLContext
from itertools import islice

from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler

from mmlspark.lightgbm import LightGBMClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import matplotlib. pyplot as plt
#creating the context
sqlContext = SQLContext(sc)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
class crash_process():
    def __init__(self, feats, categorical_columns):
        self.feats = feats
        self.categorical_columns = categorical_columns
        
    # One-hot encoding for categorical columns with get_dummies, get binary attributes
    def one_hot_encoder(self, df):
        # The index of string vlaues multiple columns
        indexers = [
            StringIndexer(inputCol=c, outputCol="{0}_indexed".format(c))
            for c in self.categorical_columns
        ]

        # The encode of indexed vlaues multiple columns
        encoders = [OneHotEncoder(dropLast=False,inputCol=indexer.getOutputCol(),
                    outputCol="{0}_encoded".format(indexer.getOutputCol())) 
            for indexer in indexers
        ]

        # Vectorizing encoded values
        assembler = VectorAssembler(inputCols=[encoder.getOutputCol() for encoder in encoders],outputCol="features")

        pipeline = Pipeline(stages=indexers + encoders+[assembler])
        model=pipeline.fit(df)
        transformed = model.transform(df)
        return transformed
    
    def preprocess(self, path, fraction = 0.8, seed = 110):
        # df1 = spark.read.csv("s3://vacarcrash/Virginia_Crashes.csv")
        #reading the virginia car crash csv file and store it in an RDD
        rdd= sc.textFile(path).map(lambda line: line.split(","))
        #removing the first row as it contains the header
        rdd = rdd.mapPartitionsWithIndex(
        lambda idx, it: islice(it, 1, None) if idx == 0 else it
        )

        #converting the RDD into a dataframe
        df = rdd.toDF(['collision', 'weather', 'vehicles', 'light', 'surface', 'location',
               'relation', 'hour', 'second', 'year', 'severity', 'ped_injury',
               'ped_fatality', 'total_injury', 'total_fatality', 'work_zone',
               'school_zone', 'route', 'district', 'minus', 'index'])
        #print the dataframe
        df = df.withColumn('hour', (round(df['hour']).astype('int')))
        
        # select features to be used
        df = df[self.feats]
        df = df[~(df['severity']== 5)]
        
        # convert severity to fatality
        exprs = [F.when(F.col('severity') == 1,1).otherwise(0).alias('fatality')]
        df = df.select(exprs+df.columns)
        df = self.one_hot_encoder(df)
        df.printSchema()
        
        #sample 20% dataset as test dataset
        (train_df, test_df) = df.randomSplit([0.8, 0.2])
        return [train_df, test_df]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
## run preprocess
feats = ['weather', 'light', 'surface','location','hour', 'severity', 'school_zone', 'route','index']
categorical_column = ['weather', 'light', 'surface','school_zone', 'route', 'location']
path = "s3://vacarcrash/"

cr = crash_process(feats, categorical_column)
train_df, test_df = cr.preprocess(path + 'Virginia_Crashes.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- fatality: integer (nullable = false)
 |-- weather: string (nullable = true)
 |-- light: string (nullable = true)
 |-- surface: string (nullable = true)
 |-- location: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- severity: string (nullable = true)
 |-- school_zone: string (nullable = true)
 |-- route: string (nullable = true)
 |-- index: string (nullable = true)
 |-- weather_indexed: double (nullable = false)
 |-- light_indexed: double (nullable = false)
 |-- surface_indexed: double (nullable = false)
 |-- school_zone_indexed: double (nullable = false)
 |-- route_indexed: double (nullable = false)
 |-- location_indexed: double (nullable = false)
 |-- weather_indexed_encoded: vector (nullable = true)
 |-- light_indexed_encoded: vector (nullable = true)
 |-- surface_indexed_encoded: vector (nullable = true)
 |-- school_zone_indexed_encoded: vector (nullable = true)
 |-- route_indexed_encoded: vector (nullable = true)
 |-- location_indexed_encoded: vector (n

In [7]:
def lgbm_vacarcrash(feature, label, learningRate=0.1,numIterations=1000,earlyStoppingRound=10):
    lgb_estimator = LightGBMClassifier(learningRate=learningRate, 
                                       numIterations=numIterations,
                                       earlyStoppingRound=earlyStoppingRound,
                                       labelCol = label)
    
    paramGrid = ParamGridBuilder().addGrid(lgb_estimator.numLeaves, [30, 50]).build()
    eval = BinaryClassificationEvaluator(labelCol = label, metricName="areaUnderROC")
    crossval = CrossValidator(estimator=lgb_estimator,
                              estimatorParamMaps=paramGrid, 
                              evaluator=eval, 
                              numFolds=3)
    cvModel  = crossval.fit(train_df[[feature, label]])
    return cvModel, paramGrid

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
(cvModel, paramGrid) = lgbm_vacarcrash("features", "fatality")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
print(list(zip(cvModel.avgMetrics, paramGrid)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(0.6705857966263649, {Param(parent='LightGBMClassifier_1ba142a0baaf', name='numLeaves', doc='numLeaves: Number of leaves (default: 31)'): 30}), (0.6695266895942336, {Param(parent='LightGBMClassifier_1ba142a0baaf', name='numLeaves', doc='numLeaves: Number of leaves (default: 31)'): 50})]

In [10]:
class CurveMetrics(BinaryClassificationMetrics):
    def __init__(self, *args):
        super(CurveMetrics, self).__init__(*args)

    def _to_list(self, rdd):
        points = []
        # Note this collect could be inefficient for large datasets 
        # considering there may be one probability per datapoint (at most)
        # The Scala version takes a numBins parameter, 
        # but it doesn't seem possible to pass this from Python to Java
        for row in rdd.collect():
            # Results are returned as type scala.Tuple2, 
            # which doesn't appear to have a py4j mapping
            points += [(float(row._1()), float(row._2()))]
        return points

    def get_curve(self, method):
        rdd = getattr(self._java_model, method)().toJavaRDD()
        return self._to_list(rdd)

bestModel = cvModel.bestModel
predictions = bestModel.transform(test_df)

# Returns as a list (false positive rate, true positive rate)
preds = predictions.select('fatality','probability').rdd.map(lambda row: (float(row['probability'][1]), float(row['fatality'])))
points = CurveMetrics(preds).get_curve('roc')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
fig = plt.figure()
x_val = [x[0] for x in points]
y_val = [x[1] for x in points]
plt.title("ROC AUC")
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.plot(x_val, y_val)
plt.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
import boto3
import json
s3 = boto3.resource('s3')
bucket_name = 'vacarcrash'
file_name = 'points'
object = s3.Object(bucket_name, file_name)
object.put(Body=(bytes(json.dumps(points, indent=2).encode('UTF-8'))))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': 'FA5F196B4DC29B84', 'HostId': 'TrcGcC9Ls5aMZvIy4SNcCv8EPKutUXvM3NdzYvZ/c/CejH4aXDAfCbGUExIjWotjbNoMwmPx3R0=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'TrcGcC9Ls5aMZvIy4SNcCv8EPKutUXvM3NdzYvZ/c/CejH4aXDAfCbGUExIjWotjbNoMwmPx3R0=', 'x-amz-request-id': 'FA5F196B4DC29B84', 'date': 'Tue, 12 May 2020 22:27:00 GMT', 'etag': '"823b6805475c56c022caa4828caa3f11"', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'ETag': '"823b6805475c56c022caa4828caa3f11"'}